# Exam

In [1]:
%pip install -q ipywidgets bqplot plotly

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import requests
import ipywidgets as widgets
from IPython.display import display, HTML
from datetime import datetime

# --- Configuration ---
ENDPOINT = "https://httpbin.org/post"  # demo endpoint, echoes what you send
NOTEBOOK_PATH = "exam.ipynb"

# --- Track last submission ---
last_submission_time = None

# --- UI Setup ---
student_code_input = widgets.Text(
    placeholder='Enter your student code',
    description='Student Code:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px')
)

submit_button = widgets.Button(
    description="📤 Submit Notebook",
    button_style="primary",
    tooltip="Send notebook to server",
)

last_submission_label = widgets.HTML(
    value="<p style='color:gray; font-style:italic;'>📅 Never submitted</p>"
)

status_output = widgets.Output()

# --- Handler ---
def on_submit_clicked(b):
    global last_submission_time
    
    with status_output:
        status_output.clear_output()
        
        # Validate student code
        student_code = student_code_input.value.strip()
        if not student_code:
            display(HTML("<p style='color:orange;'>⚠️ Please enter your student code</p>"))
            return
        
        print(f"Submitting notebook for student: {student_code}...")
        
        try:
            # Read notebook content
            with open(NOTEBOOK_PATH, "r", encoding="utf-8") as f:
                nb_content = json.load(f)
            
            # Prepare payload with student code and timestamp
            submission_time = datetime.now()
            payload = {
                "student_code": student_code,
                "notebook": nb_content,
                "timestamp": submission_time.isoformat()
            }
            
            # Send POST request
            response = requests.post(ENDPOINT, json=payload)
            
            # Check response
            if response.status_code == 200:
                last_submission_time = submission_time
                last_submission_label.value = f"<p style='color:green;'>📅 Last submitted: {submission_time.strftime('%Y-%m-%d %H:%M:%S')}</p>"
                display(HTML(f"<p style='color:green;'>✅ Submission successful for student {student_code}!</p>"))
            else:
                display(HTML(f"<p style='color:red;'>❌ Failed: {response.status_code}</p>"))
        except Exception as e:
            display(HTML(f"<p style='color:red;'>⚠️ Error: {e}</p>"))

# --- Bind the event ---
submit_button.on_click(on_submit_clicked)

# --- Display UI ---
display(
    widgets.VBox([
        widgets.HTML("<h3>📓 Notebook Submission</h3>"),
        student_code_input,
        submit_button,
        last_submission_label,
        status_output
    ])
)